In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('twcs.csv')

In [3]:
df.shape

(2811774, 7)

In [12]:
df.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 This is saddening to hear. Please shoo...,NaN,12.0
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0


In [36]:
df[df['tweet_id'].isin(list(range(11)))].sort_values(by='created_at')

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
2314174,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0
2314166,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
2314173,7,115712,True,Tue Oct 31 21:47:48 +0000 2017,@sprintcare the only way I can get a response ...,NaN,6.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0


In [8]:
df['text'][0]

'@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.'

In [9]:
df['text'][1]

'@sprintcare and how do you propose we do that'

In [15]:
df.loc[df['tweet_id'] == 2811774]

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
2639412,2811774,AmazonHelp,False,Wed Nov 22 08:08:29 +0000 2017,@784063 失礼いたします。 Amazonでのご注文商品が届かずにお困りでしょうか？その...,2811773,2811775.0


In [16]:
a = df['author_id'].tolist()

In [18]:
a = set(a)

In [22]:
names = [t for t in a if re.search('[a-zA-Z]', t)]

In [23]:
len(names)

108

In [35]:
print(df[(df['author_id'].isin(names)) & (df['inbound'] == True)].shape[0] == 0)
print(df[~(df['author_id'].isin(names)) & (df['inbound'] == False)].shape[0] == 0)

True
True


In [37]:
df_comp = df[(df['author_id'].isin(names))]
df_comp.shape

(1273931, 7)

In [78]:
agg_comp = df_comp[['tweet_id','author_id']].groupby(by=['author_id'], as_index=False).agg(['count'])
agg_comp.columns = agg_comp.columns.droplevel(0)
agg_comp = agg_comp.reset_index()
sort_df = agg_comp.sort_values(by='count')

In [47]:
x = a.author_id

MultiIndex(levels=[['tweet_id'], ['count']],
           labels=[[0], [0]])

In [57]:
a.tweet_id.count

AttributeError: 'function' object has no attribute 'columns'

In [61]:
a.tweet_id.columns

Index(['count'], dtype='object')

In [75]:
a.reset_index()

,author_id,count
0,ATT,2790
1,ATVIAssist,17650
2,AWSSupport,1035
3,AdobeCare,9917
4,AirAsiaSupport,12829
5,AirbnbHelp,8875
6,AlaskaAir,7478
7,AldiUK,8114
8,AmazonHelp,169840
9,AmericanAir,36764
